In [1]:
from itertools import product
import numpy as np
import mpmath as mp
from deepburn.CRAM import CaratheodoryFejer as CF
from deepburn.CRAM import CRAC
from tqdm.auto import tqdm
import multiprocessing as mp

In [2]:
np.set_printoptions(precision=20)

In [3]:
%%writefile CRAM_gen.py

import numpy as np
import mpmath as mp
from deepburn.CRAM import CaratheodoryFejer as CF
from deepburn.CRAM import CRAC

def tofloat(x):
    return np.array(x, dtype=float)

def tocomplex(x):
    return np.array(x, dtype=complex)

mpreal = np.vectorize(mp.re)
mpimag = np.vectorize(mp.im)

def CRAM_gen(inputs):
    dps = inputs[0]
    nf = inputs[1]
    K = inputs[2]
    order = np.arange(2,17, dtype=int)
    fname = f"CRAM_{dps}_{nf}_{K}.dat"
    with open(fname, "wt") as f:
        for n in order:
            zk, ck, rinf = CF(int(n), False, dps, K, nf)

            theta = tofloat(mpreal(zk[:(n+1)//2])) + tofloat(mpimag(zk[:(n+1)//2]))*1j
            alpha = tofloat(mpreal(ck[:(n+1)//2])) + tofloat(mpimag(ck[:(n+1)//2]))*1j
            cramdict = {
                "origin": "VandenEynde2021",
                "order": n,
                "rinf" : float(mpreal(rinf)),
                "theta": theta.tolist(),
                "alpha": alpha.tolist()
            }

            f.write(str(cramdict)+'\n')
            f.flush()

Overwriting CRAM_gen.py


In [4]:
dps_list = [15, 30, 100, 200, 400]
nf_list = [1024, 2048]
K_list = [75, 100, 150]#[75, 100, 150]

worklist = list(product(dps_list, nf_list, K_list))
print(worklist)
print(mp.cpu_count())

[(15, 1024, 75), (15, 1024, 100), (15, 1024, 150), (15, 2048, 75), (15, 2048, 100), (15, 2048, 150), (30, 1024, 75), (30, 1024, 100), (30, 1024, 150), (30, 2048, 75), (30, 2048, 100), (30, 2048, 150), (100, 1024, 75), (100, 1024, 100), (100, 1024, 150), (100, 2048, 75), (100, 2048, 100), (100, 2048, 150), (200, 1024, 75), (200, 1024, 100), (200, 1024, 150), (200, 2048, 75), (200, 2048, 100), (200, 2048, 150), (400, 1024, 75), (400, 1024, 100), (400, 1024, 150), (400, 2048, 75), (400, 2048, 100), (400, 2048, 150)]
8


In [ ]:
from CRAM_gen import CRAM_gen

for element in tqdm(worklist):
    dps = int(element[0])
    nf = int(element[1])
    K = int(element[2])
    
    with mp.Pool(processes=mp.cpu_count()) as pool:
        res = pool.map(CRAM_gen, worklist, chunksize=1+len(worklist)//mp.cpu_count())


  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
print(len(worklist))